In [1]:
from tqdm import tqdm

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import io
from PIL import Image, ImageOps
from torchvision import transforms
import torch
import cv2
import tensorflow as tf


def heatmaps_to_coordinates(heatmaps):
    """
    Heatmaps is a numpy array
    Its size - (batch_size, n_keypoints, img_size, img_size)
    """
    batch_size = heatmaps.shape[0]
    sums = heatmaps.sum(axis=-1).sum(axis=-1)
    sums = np.expand_dims(sums, [2, 3])
    normalized = heatmaps / sums
    x_prob = normalized.sum(axis=2)
    y_prob = normalized.sum(axis=3)

    arr = np.tile(np.float32(np.arange(0, 128)), [batch_size, 21, 1])
    #print(np.swapaxes(arr, 1, -1).shape)
    x = (arr * x_prob).sum(axis=2)
    y = (arr * y_prob).sum(axis=2)
    keypoints = np.stack([x, y], axis=-1)
    return keypoints / 128

def getImages(start, end, all_image_names_red, model_image_size):
    #print(str(start) + "-" + str(end))
    images_path = project_root+'/test/'
    images = np.zeros((end,model_image_size,model_image_size,3))
    for index in tqdm(range(start,end)):
      # Inputting images and resizing them
      img = cv2.imread(images_path + all_image_names_red[index])
      res = cv2.resize(img, dsize=(model_image_size, model_image_size), interpolation=cv2.INTER_CUBIC) # Resizing images
      images[index] = res
      images[index] = NormalizeData(images[index])
    return images

def normaliseImage(images):
    for i in range(3): # each color channel
      mean_img = np.mean(images[:,:,:,i])
      std_img = np.std(images[:,:,:,i])
      images[:,:,:,i] = (images[:,:,:,i] - mean_img) / std_img
    return images

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def sum_x(x):
  return tf.reduce_sum(tf.reduce_sum(x,-1),-1)

def IOUloss(y_true,y_pred):
  epsilon = 1e-6
  inter = sum_x(y_true * y_pred)
  union = (sum_x(y_true ** 2)+ sum_x(y_pred ** 2) - sum_x(y_true * y_pred))
  iou = (inter + epsilon) / (union + epsilon)
  iou = tf.reduce_mean(iou)
  return 1 - iou



In [3]:
modelpath = '/Users/ashwin-1994/Computer Vision/Final Project/models/hand_model_location_0'

unet = tf.keras.models.load_model(modelpath , custom_objects={'IOUloss': IOUloss})




2022-05-12 22:45:56.069521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-12 22:45:56.069612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [4]:
model_image_size = 128

In [16]:
# Import math Library
import math 

def predictCount(img):
    images = np.zeros((1,model_image_size,model_image_size,3))
    #img = cv2.imread('')
    res = cv2.resize(img, dsize=(model_image_size, model_image_size), interpolation=cv2.INTER_CUBIC) # Resizing images
    images[0] = res
    images[0] = NormalizeData(images[0])

    images = normaliseImage(images)

    heatmaps = unet.predict(images)

    heatmaps = np.moveaxis(heatmaps,[3],[1])

    pred_keypoints_img = heatmaps_to_coordinates(heatmaps)
    pred_keypoints_img = pred_keypoints_img[0]
    
#     print(pred_keypoints_img)

    
#   > 0.15 inc count  print(math.dist((pred_keypoints_img[8][0], pred_keypoints_img[8][1]), (pred_keypoints_img[5][0], pred_keypoints_img[5][1])))

#   > 0.25 inc count 

#   > 0.15 inc count

#   

#     print("1st")
#     print(math.dist((pred_keypoints_img[8][0], pred_keypoints_img[8][1]), (pred_keypoints_img[5][0], pred_keypoints_img[5][1])))
#     print("2nd")

#     print(math.dist((pred_keypoints_img[12][0], pred_keypoints_img[12][1]), (pred_keypoints_img[9][0], pred_keypoints_img[9][1])))
#     print("3rd")

#     print(math.dist((pred_keypoints_img[16][0], pred_keypoints_img[16][1]), (pred_keypoints_img[13][0], pred_keypoints_img[13][1])))
#     print("4th")

#     print(math.dist((pred_keypoints_img[20][0], pred_keypoints_img[20][1]), (pred_keypoints_img[17][0], pred_keypoints_img[17][1])))

    counter = 0
    
#     if (math.dist((pred_keypoints_img[8][0], pred_keypoints_img[8][1]), (pred_keypoints_img[5][0], pred_keypoints_img[5][1])) > 0.20):
#         counter += 1
#     if (math.dist((pred_keypoints_img[12][0], pred_keypoints_img[12][1]), (pred_keypoints_img[9][0], pred_keypoints_img[9][1])) > 0.25):
#         counter += 1
#     if (math.dist((pred_keypoints_img[16][0], pred_keypoints_img[16][1]), (pred_keypoints_img[13][0], pred_keypoints_img[13][1])) > 0.20):
#         counter += 1
#     if (math.dist((pred_keypoints_img[20][0], pred_keypoints_img[20][1]), (pred_keypoints_img[17][0], pred_keypoints_img[17][1])) > 0.10):
#         counter += 1
#     print("Which?")
    if ((pred_keypoints_img[8][1] > pred_keypoints_img[6][1])):
#         print("1st finger")
        counter += 1
    if ((pred_keypoints_img[12][1] > pred_keypoints_img[10][1])):
#         print("2nd finger")
        counter += 1
    if ((pred_keypoints_img[16][1] > pred_keypoints_img[14][1])):
#         print("3rd finger")
        counter += 1
#     if ((pred_keypoints_img[20][1] < pred_keypoints_img[17][1])):
#         counter += 1
#     if ((pred_keypoints_img[4][0] < pred_keypoints_img[2][0])):
#         counter += 1

    return counter

In [6]:
##########

In [7]:
import cv2
import time
import torch
import os
from PIL import Image
import numpy as np
import math
import random
from time import sleep


In [8]:
import math
import random

In [9]:
stadium = cv2.imread('/Users/ashwin-1994/Downloads/cricStadium.jpg')

In [24]:
cap = cv2.VideoCapture(0)

previousTime = 0

lst = []

numFrames = 0

playerscore = 0
botscore = 0
playerPlaying = True
matchover = False
playerCurrentScore = 0
botCurrentScore = 0
whoIsPlaying = 'Player'
whoIsOut = ''
whoWins = ''

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_copy = img.copy()
    currentTime = time.time()
    fps = 1/(currentTime-previousTime)
    previousTime = currentTime
    img[ 100:559 , 650:1262 ] = stadium
    
    cv2.putText(img, f'FPS: {int(fps)}', (30, 50), cv2.FONT_HERSHEY_PLAIN,
                1.5, (255, 0, 0), 3)
    
    cv2.putText(img, '!!! Hand Cricket !!!', (800, 150), cv2.FONT_HERSHEY_PLAIN,
                2, (255, 255, 0), 3)
    cv2.putText(img, f'PLayer Score: {int(playerscore)}', (750, 200), cv2.FONT_HERSHEY_PLAIN,
                1.7, (255, 255, 0), 3)
    cv2.putText(img, f'Bot Score: {int(botscore)}', (1000, 200), cv2.FONT_HERSHEY_PLAIN,
                1.7, (255, 255, 0), 3)
    cv2.putText(img, f'{str(whoIsPlaying)} Batting !!', (750, 250), cv2.FONT_HERSHEY_PLAIN,
                1.7, (255, 255, 0), 3)
    cv2.putText(img, 'Previous Play', (750, 300), cv2.FONT_HERSHEY_PLAIN,
                1.5, (255, 0, 0), 3)
    cv2.putText(img, f'Player played: {int(playerCurrentScore)}', (750, 350), cv2.FONT_HERSHEY_PLAIN,
                1.5, (255, 255, 0), 3)
    cv2.putText(img, f'Bot played: {int(botCurrentScore)}', (1020, 350), cv2.FONT_HERSHEY_PLAIN,
                1.5, (255, 255, 0), 3)
    cv2.putText(img, f'{str(whoIsOut)}', (750, 400), cv2.FONT_HERSHEY_PLAIN,
                1.5, (0, 0, 255), 3)
    
    cv2.putText(img, f'{str(whoWins)}', (750, 450), cv2.FONT_HERSHEY_PLAIN,
                1.5, (0, 255, 0), 3)
    
    

    '''rectangle -> frame,start point,end point,color,thickness'''
    cv2.rectangle(img, (100, 100), (600, 600), (0, 255, 0), 2)

    clipped_image = img[100:600,100:600]
    (h,w) = clipped_image.shape[:2]

    '''send clipped image for further processing'''
    res = predictCount(clipped_image)
    lst.append(res)
    cv2.putText(img, f'Count: {int(res)}', (160, 50), cv2.FONT_HERSHEY_PLAIN,
                1.5, (255, 0, 0), 3)
    cv2.imshow("Image",img)


    '''Close cam window if we press z'''
    camclose = cv2.waitKey(1) & 0xFF
    
    if camclose == ord("z"):
        break
    numFrames += 1
    if numFrames == 70:
        playerCurrentScore = (max(set(lst[10:]), key=lst.count))
        cv2.putText(img, f'count: {int(playerCurrentScore)}', (100, 100), cv2.FONT_HERSHEY_PLAIN,
                1.5, (255, 0, 0), 3)
        
        botCurrentScore = random.randint(0,5)
        
        if playerPlaying:
            whoIsPlaying = 'Player'
            print('Player playing')
            print('Player played:')
            print(playerCurrentScore)
            print('Bot played:')
            print(botCurrentScore)
            if playerCurrentScore == botCurrentScore:
                print('Player out')
                whoIsOut = 'PLayer is OUT'
                playerPlaying = False
                print('')
                print('')
                if playerCurrentScore != 0:
                    botCurrentScore = 0 
                else:
                    playerCurrentScore = 1
                    botCurrentScore = 0
            else:
                playerscore += playerCurrentScore
                print('Player Score = '+ str(playerscore))
                print('Bot Score ='+ str(botscore))
            
            
        else:
            whoIsPlaying = 'Bot'
            print('Bot playing')
            print('Player played:')
            print(playerCurrentScore)
            print('Bot played:')
            print(botCurrentScore)
            if playerCurrentScore == botCurrentScore:
                print('Bot out')
                whoIsOut = 'Bot is OUT'
                if playerscore > botscore:
                    print('Player Wins')
                    whoWins = 'PLayer WINS !!!!'
                elif playerscore < botscore:
                    print('Bot Wins')
                    whoWins = 'Bot WINS !!!!'
                else:
                    print('Match Tied')
                    whoWins = 'Match Tied !!!!'
                matchover = True
            
            else:
                botscore += botCurrentScore
                print('Player Score = '+ str(playerscore))
                print('Bot Score ='+ str(botscore))
            
            if playerscore < botscore:
                print('Bot Wins')
                whoWins = 'Bot WINS !!!!'
                matchover = True
                
            if matchover:
                print('Total Match scores:')
                print('Player Score = '+ str(playerscore))
                print('Bot Score ='+ str(botscore))
                
                playerscore = 0
                botscore = 0
                playerPlaying = True
                matchover = False
                
        camclose = cv2.waitKey(-1)
        
    if camclose == ord("r"):
        lst = []
        #print(lst)
        numFrames = 0
        #print(numFrames)
        camclose = cv2.waitKey(1) & 0xFF
        
cap.release()
cv2.destroyAllWindows()

Player playing
Player played:
1
Bot played:
3
Player Score = 1
Bot Score =0
Player playing
Player played:
0
Bot played:
1
Player Score = 1
Bot Score =0
Player playing
Player played:
1
Bot played:
5
Player Score = 2
Bot Score =0
Player playing
Player played:
1
Bot played:
0
Player Score = 3
Bot Score =0
Player playing
Player played:
1
Bot played:
5
Player Score = 4
Bot Score =0
Player playing
Player played:
2
Bot played:
4
Player Score = 6
Bot Score =0
Player playing
Player played:
0
Bot played:
3
Player Score = 6
Bot Score =0
Player playing
Player played:
1
Bot played:
5
Player Score = 7
Bot Score =0
Player playing
Player played:
1
Bot played:
0
Player Score = 8
Bot Score =0
Player playing
Player played:
1
Bot played:
1
Player out


Bot playing
Player played:
1
Bot played:
1
Bot out
Player Wins
Total Match scores:
Player Score = 8
Bot Score =0
Player playing
Player played:
1
Bot played:
4
Player Score = 1
Bot Score =0


In [11]:
cap.release()